In [140]:
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import LoadJobConfig
from pandas_gbq import to_gbq
import pandas as pd

import re

from google_auth_oauthlib import flow

### TEST

In [121]:
scopes=["https://www.googleapis.com/auth/cloud-platform"]
bq_credentials = './credenciales/client_secret_python_bq.json'
project_id = 'curso-bigquery-mide-403114'
dataset_id = 'bigquery-public-data'
table_id = 'ga4_obfuscated_sample_ecommerce.events_20210131'
table_ref = client.dataset(dataset_id).table(table_id)

In [122]:
try:
    credentials = service_account.Credentials.from_service_account_file(bq_credentials, scopes=scopes)
    client = bigquery.Client(credentials=credentials, project=credentials.project_id)
except Exception as e:
    print("Big query conection wrong: " + str(e))

In [126]:
#query de ejemplo
query_ejemplo = """
    SELECT *
    FROM `{}.{}`
    Limit 10
""".format(dataset_id,table_id)

query_ejecutada_ejemplo = client.query(query_ejemplo)
resultado_ejemplo = query_ejecutada_ejemplo.result()

#for row_ejemplo in resultado_ejemplo:
#    print(row_ejemplo)

-----------
#### CREAR TABLA E INSERTAR DATOS

#### configuración

In [147]:
#VARIABLES
scopes=["https://www.googleapis.com/auth/cloud-platform"]
bq_credentials = './credenciales/client_secret_python_bq.json'

project_id = 'curso-bigquery-mide-403114'
dataset_id = 'chicago_taxi_tips'
table_id = 'cars_upload_test'
csv_path = './files/cars.csv'

In [148]:
#CONEXION
def bq_client(scopes, bq_credentials, project_id):
    try:
        credentials = service_account.Credentials.from_service_account_file(bq_credentials, scopes=scopes)
        client = bigquery.Client(credentials=credentials, project=credentials.project_id)
        return client
    except Exception as e:
        print("Big query conection wrong: " + str(e))

client = bq_client(scopes, bq_credentials, project_id)

In [149]:
#REFERENCIAR LA TABLA
def bq_table_ref(client,dataset_id,table_id):
    try:
        table_ref = client.dataset(dataset_id).table(table_id)
        return table_ref
    except Exception as e:
        print("Error in Bigquery table reference: " + str(e))

bq_table_ref = bq_table_ref(client,dataset_id,table_id)

In [150]:
#CSV a un DataFrame de Pandas
def bq_csv_to_pandas(csv_path):
    try:
        bigquery_csv = pd.read_csv(csv_path, sep=",")
        return bigquery_csv
    except Exception as e:
        print("Error importing the csv file" + str(e))

bq_csv_df = bq_csv_to_pandas(csv_path)

##### a) SQL

In [177]:
#correr después de crear la query:
def bq_query_execution(client, query, dataset_id, table_id):
    try:
        query_ejecutada = client.query(query)
        resultado_query = query_ejecutada.result()

        if "CREATE TABLE" in query:
            return("Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id))
        elif "INSERT" in query:
            return("Values inserted into {}.{}.{}".format(table.project, table.dataset_id, table.table_id))
        elif "TRUNCATE TABLE" in query:
            return("TRUNCATE {}.{}.{}".format(table.project, table.dataset_id, table.table_id))
        else:
            pass
        
    except Exception as e:
        print(e)

###### 1.A crear la tabla si no existe

In [179]:
#CREAR ESQUEMA DESDE EL CSV 

def bq_schema_for_query(bq_csv_df):
    schema_for_query = {}

    # Itera sobre las columnas y determina el tipo de datos
    try:
        for column in bigquery_csv_df.columns:
            # Infiere el tipo de datos basándose en los tipos de pandas
            if pd.api.types.is_string_dtype(bq_csv_df[column]):
                schema_for_query[column] = 'STRING'
            elif pd.api.types.is_numeric_dtype(bq_csv_df[column]):
                if pd.api.types.is_float_dtype(bq_csv_df[column]):
                    schema_for_query[column] = 'FLOAT'
                else:
                    schema_for_query[column] = 'INTEGER'
            else:
                schema_for_query[column] = 'STRING'
                
        schema_for_query = re.sub("'|\{|\}|\[|\]|\:","",str(schema))
        schema_for_query = re.sub("FLOAT","FLOAT64",str(schema_for_query))
        schema_for_query = re.sub("INTEGER","INT64",str(schema_for_query))

        return schema_for_query
        
    except Exception as e:
        print(e)

bq_schema_for_query = bq_schema_for_query(bigquery_csv_df)

In [180]:
#crear la tabla si no existe --> https://cloud.google.com/bigquery/docs/reference/standard-sql/data-definition-language
def bq_query_create_table(dataset_id,table_id,bq_schema_for_query):
    query = """CREATE TABLE IF NOT EXISTS `{}.{}`({});""".format(dataset_id,table_id,bq_schema_for_query)
    return query
    
    #PARTITION BY fecha
    #OPTIONS(
    #    partition_expiration_days=100,
    #    description="a table partitioned by fecha");

bq_query_create_table = bq_query_create_table(dataset_id,table_id,bq_schema_for_query)

In [176]:
#ejecución de la query
bq_query_execution(client, bq_query_create_table, dataset_id, table_id)

'Created table curso-bigquery-mide-403114.chicago_taxi_tips.cars_upload_test'

###### 2.A INSERTAR datos del csv en la tabla debajo de los ya existentes

In [189]:
def bq_query_insert_values(bq_csv_df,dataset_id,table_id):
    #crear columnas
    columns_for_query = list(bq_csv_df.columns)
    columns_for_query = re.sub("'|\{|\}|\[|\]|\:","",str(columns_for_query))

    #crear valores
    values_for_query = str([tuple(i_value) for i_value in bq_csv_df.values.tolist()])
    values_for_query = re.sub("\[|\]","",str(values_for_query))

    #crear query
    query = """INSERT `{}.{}`({}) VALUES {}""".format(dataset_id,table_id,columns_for_query,values_for_query)
    return query

bq_query_insert_values = bq_query_insert_values(bq_csv_df,dataset_id,table_id)

In [186]:
bq_query_execution(client, bq_query_insert_values, dataset_id, table_id)

403 Billing has not been enabled for this project. Enable billing at https://console.cloud.google.com/billing. DML queries are not allowed in the free tier. Set up a billing account to remove this restriction.

Location: europe-southwest1
Job ID: 949cd01c-0ffc-4689-9755-d61d506eaa23



###### * Si queremos eliminar los datos antes de insertarlos tendríamos que hacer un TRUNCATE

In [190]:
def bq_query_truncate_table(dataset_id,table_id):
    query = """TRUNCATE TABLE `{}.{}`""".format(dataset_id,table_id)
    return query

bq_query_truncate_table = bq_query_truncate_table(dataset_id,table_id)

In [191]:
bq_query_execution(client, bq_query_truncate_table, dataset_id, table_id)

403 Billing has not been enabled for this project. Enable billing at https://console.cloud.google.com/billing. DML queries are not allowed in the free tier. Set up a billing account to remove this restriction.

Location: europe-southwest1
Job ID: 43b89f91-831d-46bd-8805-2e1d22c880d7



##### b) python
https://cloud.google.com/bigquery/docs/tables?hl=es-419

###### 1.A crear la tabla a mano si no existe

In [16]:
# si quiero añadir particionar tablas:  https://cloud.google.com/bigquery/docs/creating-partitioned-tables?hl=es-419

project_id = 'curso-bigquery-mide-403114'
dataset_id = 'chicago_taxi_tips'
table_id = 'cars_upload_test'

try:
    credentials = service_account.Credentials.from_service_account_file(bq_credentials, scopes=scopes)
    client = bigquery.Client(credentials=credentials, project=credentials.project_id)
except Exception as e:
    print("Big query conection wrong: " + str(e))

try:
    table_ref = client.dataset(dataset_id).table(table_id)
except Exception as e:
    print("Error in Bigquery table reference: " + str(e))

try:
    client.get_table(table_ref)
    print(f"La tabla {table_id} ya existe en el conjunto de datos {dataset_id}.")
except Exception as e:
    if "Not found" in str(e):
        print(f"La tabla {table_id} no existe en el conjunto de datos {dataset_id}.")
        # Crea la tabla si no existe
        schema = [
            bigquery.SchemaField("cars", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("mpg", "FLOAT", mode="REQUIRED"),
            bigquery.SchemaField("cyl", "INTEGER", mode="NULLABLE"),
            bigquery.SchemaField("disp", "FLOAT", mode="NULLABLE"),
            bigquery.SchemaField("hp", "INTEGER", mode="NULLABLE"),
            bigquery.SchemaField("drat", "FLOAT", mode="NULLABLE"),
            bigquery.SchemaField("wt", "FLOAT", mode="NULLABLE"),
            bigquery.SchemaField("qsec", "FLOAT", mode="NULLABLE"),
            bigquery.SchemaField("vs", "INTEGER", mode="NULLABLE"),
            bigquery.SchemaField("am", "INTEGER", mode="NULLABLE"),
            bigquery.SchemaField("gear", "INTEGER", mode="NULLABLE"),
            bigquery.SchemaField("carb", "INTEGER", mode="NULLABLE")
        ]
        table = bigquery.Table(table_ref, schema=schema)

        #para crear tabla particionada
        #table.time_partitioning = bigquery.TimePartitioning(
        #    type_=bigquery.TimePartitioningType.DAY,
        #    field="date",  # name of column to use for partitioning
        #    expiration_ms=1000 * 60 * 60 * 24 * 90,
        #)  # 90 days
        
        table = client.create_table(table)
        print("Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id))
    else:
        print(f"Error: {e}")

La tabla cars_upload_test ya existe en el conjunto de datos chicago_taxi_tips.


###### 1.B crear la tabla leyendo el schema del csv

In [28]:
csv_path = './files/cars.csv'

# Crea una instancia de la tabla en BigQuery (no la crea, solo define el esquema)
project_id = 'curso-bigquery-mide-403114'
dataset_id = 'chicago_taxi_tips'
table_id = 'cars_upload_test'

try:
    table_ref = client.dataset(dataset_id).table(table_id)
except Exception as e:
    print("Error in Bigquery table reference: " + str(e))

# Carga una muestra del CSV en un DataFrame de Pandas
try:
    sample_df = pd.read_csv(csv_path, sep=",")
except Exception as e:
    print("Error importing the csv file" + str(e))
# Crea un diccionario para almacenar el esquema inferido
schema = {}

# Itera sobre las columnas y determina el tipo de datos
try:
    for column in sample_df.columns:
        # Infiera el tipo de datos basándose en los tipos de pandas
        if pd.api.types.is_string_dtype(sample_df[column]):
            schema[column] = 'STRING'
        elif pd.api.types.is_numeric_dtype(sample_df[column]):
            if pd.api.types.is_float_dtype(sample_df[column]):
                schema[column] = 'FLOAT'
            else:
                schema[column] = 'INTEGER'
        else:
            schema[column] = 'STRING'  # Puedes ajustar según sea necesario
except Exception as e:
    print(e)

# iterar este esquema para crear el formato qeu BQ necesita en un listado (ejemplo mas arriba en la creacion a mano del listado)
try:
    schema2=[bigquery.SchemaField(keyx, valuex, mode="NULLABLE") for keyx,valuex in schema.items()]
except Exception as e:
    print(e)
    
try:
    client.get_table(table_ref)
    print(f"La tabla {table_id} ya existe en el conjunto de datos {dataset_id}.")
except Exception as e:
    if "Not found" in str(e):
        print(f"La tabla {table_id} no existe en el conjunto de datos {dataset_id}.")
        
        # Crea la tabla si no existe
        table = bigquery.Table(table_ref, schema=schema2)

        #para crear tabla particionada
        #table.time_partitioning = bigquery.TimePartitioning(
        #    type_=bigquery.TimePartitioningType.DAY,
        #    field="date",  # name of column to use for partitioning
        #    expiration_ms=1000 * 60 * 60 * 24 * 90,
        #)  # 90 days
        
        table = client.create_table(table)
        print("Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id))

La tabla cars_upload_test no existe en el conjunto de datos chicago_taxi_tips.
Created table curso-bigquery-mide-403114.chicago_taxi_tips.cars_upload_test


###### 2.A INSERTAR datos del csv en la tabla debajo de los ya existentes

In [29]:
#cargar datos una vez que ya existe (los inserta debajo, si queremos sustituir, habrá que eliminarlos antes)
csv_path ='./files/cars.csv'

# Carga el CSV en un DataFrame de Pandas
try:
    df = pd.read_csv(csv_path, sep=",")
except Exception as e:
    print("Error importing the csv file" + str(e))

# Crea una instancia de la tabla en BigQuery (está hecho arriba, si lo hacemos junto habrá que eliminar las 2 siguientes lineas))
try:
    table_ref = client.dataset(dataset_id).table(table_id)
    table = bigquery.Table(table_ref)
except Exception as e:
    print("Error in Bigquery table reference: " + str(e))

try:
    client.get_table(table_ref)
    print(f"La tabla {table_id} existe en el conjunto de datos {dataset_id}.")

    job = client.load_table_from_dataframe(df, table)
    job.result()  # Espera a que se complete la carga

    print(f"Datos del CSV cargados en la tabla {table_id} de BigQuery.")
except:
    print(f"La tabla {table_id} NO existe en el conjunto de datos {dataset_id}, hay que crearla")

La tabla cars_upload_test existe en el conjunto de datos chicago_taxi_tips.
Datos del CSV cargados en la tabla cars_upload_test de BigQuery.


###### 2.B REEMPLAZAR datos ya existentes en la tabla por los del csv

In [14]:
#cargar datos una vez que ya existe (los inserta debajo, si queremos sustituir, habrá que eliminarlos antes)
csv_path ='./files/cars.csv'

# Carga el CSV en un DataFrame de Pandas
try:
    df = pd.read_csv(csv_path, sep=",", index_col=False)
except Exception as e:
    print("Error importing the csv file" + str(e))

# Crea una instancia de la tabla en BigQuery (está hecho arriba, si lo hacemos junto habrá que eliminar las 2 siguientes lineas))
try:
    table_ref = client.dataset(dataset_id).table(table_id)
    table = bigquery.Table(table_ref)
except Exception as e:
    print("Error in Bigquery table reference: " + str(e))

try:
    client.get_table(table_ref)
    print(f"La tabla {table_id} existe en el conjunto de datos {dataset_id}.")
    
    job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE")
    job = client.load_table_from_dataframe(df, table, job_config=job_config)
    job.result()  # Espera a que se complete la carga

    print(f"Datos del CSV reemplazaron los existentes en la tabla {table_id} de BigQuery.")

except:
    print(f"La tabla {table_id} NO existe en el conjunto de datos {dataset_id}, hay que crearla")

La tabla cars_upload_test existe en el conjunto de datos chicago_taxi_tips.
Datos del CSV reemplazaron los existentes en la tabla cars_upload_test de BigQuery.


##### c) pandas_gbq  -> requiere autenticación

In [15]:
project_id = 'curso-bigquery-mide-403114'
dataset_id = 'chicago_taxi_tips'
table_id = 'cars_upload_test'

try:
    credentials = service_account.Credentials.from_service_account_file(bq_credentials, scopes=scopes)
    client = bigquery.Client(credentials=credentials, project=credentials.project_id)
except Exception as e:
    print("Big query conection wrong: " + str(e))

# Ruta al archivo CSV en tu sistema local
csv_path = './files/cars.csv'  # Reemplaza con tu ruta real

# Verifica si la tabla existe en el conjunto de datos
try:
    table_ref = client.dataset(dataset_id).table(table_id)
except Exception as e:
    print("Error in Bigquery table reference: " + str(e))
    
try:
    client.get_table(table_ref)
    print(f"La tabla {table_id} ya existe en el conjunto de datos {dataset_id}.")
except Exception as e:
    if "Not found" in str(e):
        print(f"La tabla {table_id} no existe en el conjunto de datos {dataset_id}.")

        # Carga el CSV en un DataFrame de Pandas
        df = pd.read_csv(csv_path, sep=",", index_col=False)

        # Carga el DataFrame en BigQuery
        to_gbq(df, f'{project_id}.{dataset_id}.{table_id}', project_id=project_id, if_exists='replace')

        print(f"La tabla {table_id} ha sido creada y los datos del CSV han sido cargados en BigQuery.")
    else:
        print(f"Error: {e}")

La tabla cars_upload_test ya existe en el conjunto de datos chicago_taxi_tips.
